In [135]:
#Restrições
# Um vértice pode representar no máximo uma partição
# Uma partição pode ter no máximo um representante
# Uma partição deve ter como representante o vértice de menor índice
# Desvio absoluto do número de partições em relação ao valor K
# Contendo o valor máximo das variáveis de folga
# Se um vértice A é ligado à um vértice B então A deve ser vizinho de todos
#   os vizinhos de B.

In [ ]:
#TODO
#A terceira restrição precisa de correções
#Algo que englobe o representante de menor indice da outra partição
#Se a partição existe
#Se o índice selecionado é o menor possível dado que os menores que ele 
#  já estão sendo utilizados em outras partições

$$
\begin{align}
V: & \text{set of vertices indices}\\
E: & \text{set of edges indices}\\
\Pi: & \text{set of partitions indices}\\
K: & \text{number of partitions } |\Pi|\\
\rho_{i\pi}:  & \text{representative vertex $i$ of the partition $\pi$}\\
\epsilon_{ij}:  & \text{edge between the vertices $i$ and $j$}\\
s^-, s^+: & \text{absolute deviation from the number of partitions}\\
\omega_{ij}:  & \text{wieght of the edge between the vertices $i$ and $j$}\\
n(i): & \text{neighborhoods of the edge $i$}\\
\varphi: & \text{penalty for not create a partition}
\end{align}
$$


$$
\underset{\rho, \epsilon, s}\min \left\{z := \sum_{(i, j) \in E}\omega_{ij}\left[\epsilon_{ij} + (1 + \lambda)(1 - \epsilon_{ij})\right] + \varphi\big(K - s^- - s^+\big) \right\}, \text{ where } \varphi := \sum_{(i, j) \in E}\omega_{ij}, \lambda \in (0, 1]
$$

$$
\begin{align}
\sum_{\pi \in \Pi} \rho_{i\pi} \leq 1 & & \forall i \in V\\
\sum_{i \in V} \rho_{i\pi} \leq 1 & & \forall \pi \in \Pi\\
\alpha\rho_{\alpha\pi} - \sum_{u \in \Pi}\left[\alpha\rho_{\beta u} + \beta(1 - \rho_{\beta u})\right] \leq \rho_{\alpha\pi} - \sum_{\gamma \in V}\rho_{\gamma\pi} & & \forall (\alpha, \beta, \pi) \in V \times V \times \Pi\\
\sum_{\pi \in \Pi}\sum_{i \in V} \rho_{i\pi} + s^- - s^+ = K & &\\
s^- + s^+ \leq K\\
\epsilon_{ij}\sum_{k \in n(i) \cap n(j)}\big(\epsilon_{ik} + \epsilon_{jk}\big) = \sum_{k \in n(i)}\epsilon_{ik} + \sum_{k \in n(j)}\epsilon_{jk} - 2 & & \forall (i, j) \in E\\
\rho_{i\pi} \in \{0, 1\} & & \forall (i, \pi) \in V \times \Pi\\
\epsilon_{ij} \in \{0, 1\} & & \forall (i, j) \in E\\
s^-, s^+ \in \mathbb{R}^+
\end{align}
$$

### Linearized constraint (5)

$$
(|n(i)| + |n(j)|)\left(1 - \epsilon_{ij}\right) + \sum_{k \in n(i) \cap n(j)}\big(\epsilon_{ik} + \epsilon_{jk}\big) \geq \sum_{k \in n(i)}\epsilon_{ik} + \sum_{k \in n(j)}\epsilon_{jk} - 2
$$

### Constraint (3)

$$
\alpha\rho_{\alpha\pi} \leq \sum_{\pi \in \Pi}\left(\beta(1 - \rho_{\beta\pi}) + \alpha\rho_{\beta\pi}\right)
$$


$$
\alpha\rho_{\alpha\pi} - \sum_{\pi \in \Pi}\left[\alpha\rho_{\beta\pi} + \beta(1 - \rho_{\beta\pi})\right] \leq \alpha\sum_{\gamma \in V}\rho_{\gamma\pi}
$$

## Testing constraint 5

In [136]:
import numpy as np

def remove_edges(edges):
    for i, j in edges:
        graph[i, j] = 0
        graph[j, i] = 0
        epsilon[i, j] = 0
        epsilon[j, i] = 0
        
def disable_edges(edges):
    for i, j in edges:
        epsilon[i, j] = 0
        epsilon[j, i] = 0
    
def test_constraint(graph, epsilon):
    def fn(i, j):
        indices = np.arange(0, len(graph))
        n_i = indices[graph[i] != 0]
        n_j = indices[graph[j] != 0]
        
        n_ij = indices[(graph[i] != 0) & (graph[j] != 0)]
        
        lhs = epsilon[i, j] * (epsilon[i, n_ij].sum() + epsilon[j, n_ij].sum())
        rhs = epsilon[i, n_i].sum() + epsilon[j, n_j].sum() - 2
        
        return f'{lhs} = {rhs}   {lhs == rhs}'
        
    for i in range(len(graph)):
        for j in range(len(graph)):
            if graph[i, j] != 0 and graph[j, i] != 0:
                print((i, j), fn(i, j))

### Complete graph (must be feasible)

In [137]:
graph = -(np.eye(4, dtype=int) - 1) 
epsilon = -(np.eye(4, dtype=int) - 1) 

print('Graph\n', '\n'.join(map(str, graph)), sep='', end='\n\n')
print('Epsilon\n', '\n'.join(map(str, epsilon)), sep='', end='\n\n')
            
test_constraint(graph, epsilon)

Graph
[0 1 1 1]
[1 0 1 1]
[1 1 0 1]
[1 1 1 0]

Epsilon
[0 1 1 1]
[1 0 1 1]
[1 1 0 1]
[1 1 1 0]

(0, 1) 4 = 4   True
(0, 2) 4 = 4   True
(0, 3) 4 = 4   True
(1, 0) 4 = 4   True
(1, 2) 4 = 4   True
(1, 3) 4 = 4   True
(2, 0) 4 = 4   True
(2, 1) 4 = 4   True
(2, 3) 4 = 4   True
(3, 0) 4 = 4   True
(3, 1) 4 = 4   True
(3, 2) 4 = 4   True


### Edge (0, 2) does not exists (must be infeasible)

In [138]:
graph = -(np.eye(4, dtype=int) - 1) 
epsilon = -(np.eye(4, dtype=int) - 1) 

remove_edges([(0, 2)])

print('Graph\n', '\n'.join(map(str, graph)), sep='', end='\n\n')
print('Epsilon\n', '\n'.join(map(str, epsilon)), sep='', end='\n\n')
            
test_constraint(graph, epsilon)

Graph
[0 1 0 1]
[1 0 1 1]
[0 1 0 1]
[1 1 1 0]

Epsilon
[0 1 0 1]
[1 0 1 1]
[0 1 0 1]
[1 1 1 0]

(0, 1) 2 = 3   False
(0, 3) 2 = 3   False
(1, 0) 2 = 3   False
(1, 2) 2 = 3   False
(1, 3) 4 = 4   True
(2, 1) 2 = 3   False
(2, 3) 2 = 3   False
(3, 0) 2 = 3   False
(3, 1) 4 = 4   True
(3, 2) 2 = 3   False


### Edge (0, 2), (2, 1) and (2, 3) does not exists (must be feasible)

In [139]:
graph = -(np.eye(4, dtype=int) - 1) 
epsilon = -(np.eye(4, dtype=int) - 1) 

remove_edges([(0, 2)])
disable_edges([(2, 1), (2, 3)])

print('Graph\n', '\n'.join(map(str, graph)), sep='', end='\n\n')
print('Epsilon\n', '\n'.join(map(str, epsilon)), sep='', end='\n\n')
            
test_constraint(graph, epsilon)

Graph
[0 1 0 1]
[1 0 1 1]
[0 1 0 1]
[1 1 1 0]

Epsilon
[0 1 0 1]
[1 0 0 1]
[0 0 0 0]
[1 1 0 0]

(0, 1) 2 = 2   True
(0, 3) 2 = 2   True
(1, 0) 2 = 2   True
(1, 2) 0 = 0   True
(1, 3) 2 = 2   True
(2, 1) 0 = 0   True
(2, 3) 0 = 0   True
(3, 0) 2 = 2   True
(3, 1) 2 = 2   True
(3, 2) 0 = 0   True
